In [1]:
import os
import numpy as np
import itertools
import collections
import networkx as nx

In [2]:
def aoc_2021_19_1(file_path):
    """--- Day 19: Beacon Scanner --- Part One"""

    def scanner_transformation(i_scanner0, i_scanner1):

        beacon0_beacon1_pos = {}
        beacon0_pos = []
        beacon1_pos = []
        beacon0_set = set()
        beacon1_set = set()
        
        # read beacon distance & positions
        beacon_distance_0 = beacon_distance_list[i_scanner0]
        beacon_distance_1 = beacon_distance_list[i_scanner1]
        beacon_position_0 = beacon_position_list[i_scanner0]
        beacon_position_1 = beacon_position_list[i_scanner1]
        
        # shared distance between beacons
        for shared_dist in sorted(set(beacon_distance_0).intersection(set(beacon_distance_1))):
            temp_beacon0 = beacon_distance_0[shared_dist]
            temp_beacon1 = beacon_distance_1[shared_dist]
            beacon0_pos.append(temp_beacon0)
            beacon1_pos.append(temp_beacon1)
            beacon0_set.update(temp_beacon0)
            beacon1_set.update(temp_beacon1)
        
        # find positions of shared beacon
        for beacon0 in sorted(beacon0_set):
            temp_beacon1 = [beacon1_pos[i_b] for i_b, b in enumerate(beacon0_pos) if beacon0 in b]
            beacon1 = set.intersection(*temp_beacon1)

            try:
                beacon0_beacon1_pos[beacon0] = list(beacon1)[0]
            except IndexError:
                temp_counter = collections.Counter(list(itertools.chain.from_iterable(temp_beacon1)))
                temp_counter_max = max(temp_counter.values())
                beacon0_beacon1_pos[beacon0] = [k for k, v in temp_counter.items() if v == temp_counter_max][0]

        beacon0_pos_array = np.array([beacon_position_0[b] for b in sorted(beacon0_beacon1_pos)])
        beacon1_pos_array = np.array([beacon_position_1[beacon0_beacon1_pos[b]] for b in sorted(beacon0_beacon1_pos)])

        # calculate transformations
        transformation_tuple = None
        for i_perspective, perspective in enumerate(perspective_list):
            temp_scanner_pos = set([tuple(b) for b in beacon0_pos_array - np.dot(beacon1_pos_array, perspective)])
            if len(temp_scanner_pos) == 1:
                transformation_tuple = (i_perspective, list(temp_scanner_pos)[0])
                break

        if transformation_tuple is None:
            duplicate_scanner_limit = 2
            while duplicate_scanner_limit < 12 and transformation_tuple is None:
                for i_perspective, perspective in enumerate(perspective_list):
                    temp_scanner_list = [tuple(b) for b in beacon0_pos_array - np.dot(beacon1_pos_array, perspective)]
                    temp_scanner_pos = set(temp_scanner_list)
                    if len(temp_scanner_pos) == duplicate_scanner_limit:
                        temp_scanner_counter = collections.Counter(temp_scanner_list)    
                        transformation_tuple = (i_perspective, [k for k in temp_scanner_pos if temp_scanner_counter[k] == max(temp_scanner_counter.values())][0])
                        break
                duplicate_scanner_limit += 1

        if transformation_tuple is None:
            raise ValueError

        return transformation_tuple

    def transform_all_beacons(last_scanner, this_scanner):
        """Calculate all beacon positions"""

        all_current_beacons = beacon_position_list[this_scanner]
        dfs_dict = nx.algorithms.traversal.depth_first_search.dfs_successors(scanner_graph, 0)
        
        # dfs all scanners
        if this_scanner in dfs_dict:
            for next_scanner in dfs_dict[this_scanner]:
                current_transformation = scanner_transformation(this_scanner, next_scanner)
                temp_beacons = current_transformation[1] + np.dot(transform_all_beacons(this_scanner, next_scanner), perspective_list[current_transformation[0]])
                all_current_beacons += [b for b in temp_beacons]

        all_current_beacons = np.array(sorted(set([tuple(b) for b in all_current_beacons])))

        return all_current_beacons
    
    # main function
    with open(file_path) as f:
        aoc_read = f.read().split('\n\n')

    beacon_distance_list = []
    beacon_position_list = []
    
    # define all 24 perspectives
    temp_perm = [np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
                 np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]]),
                 np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]]),
                 np.array([[-1, 0, 0], [0, -1, 0], [0, 0, 1]]),
                 np.array([[0, -1, 0], [0, 0, -1], [1, 0, 0]]),
                 np.array([[0, 0, -1], [-1, 0, 0], [0, 1, 0]])]
    temp_rot = [np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
                np.array([[1, 0, 0], [0, 0, -1], [0, 1, 0]]),
                np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]]),
                np.array([[1, 0, 0], [0, 0, 1], [0, -1, 0]])]
    perspective_list = [np.dot(p[1], p[0]) for p in itertools.product(temp_perm, temp_rot)]
    
    # read beacon distances for each scanner
    for i_scanner, scanner_read in enumerate(aoc_read):
        beacon_list = [np.array([int(b) for b in beacon.split(',')]) for beacon in scanner_read.split('\n')[1:] if beacon]
        beacon_position_list.append(beacon_list)

        distance_dict = {}
        for i_beacon0 in range(len(beacon_list) - 1):
            for i_beacon1 in range(i_beacon0 + 1, len(beacon_list)):
                temp_distance = np.around(np.sqrt(np.sum(np.square(beacon_list[i_beacon0] - beacon_list[i_beacon1]))), decimals=5)
                distance_dict[temp_distance] = {i_beacon0, i_beacon1}

        beacon_distance_list.append(distance_dict)

    # connect scanners by common beacons
    scanner_graph = nx.Graph()
    for i_scanner0 in range(len(beacon_position_list)):
        for i_scanner1 in range(len(beacon_position_list)):
            if i_scanner0 == i_scanner1:
                continue
            if len(set(beacon_distance_list[i_scanner0]).intersection(set(beacon_distance_list[i_scanner1]))) >= 66:
                scanner_graph.add_edge(i_scanner0, i_scanner1)

    # calculate all beacons
    no_beacons = len(transform_all_beacons(0, 0))

    return no_beacons

In [3]:
aoc_2021_19_1('example.txt')

79

In [4]:
aoc_2021_19_1('input.txt')

335

In [5]:
def aoc_2021_19_2(file_path):
    """--- Day 19: Beacon Scanner --- Part Two"""

    def scanner_transformation(i_scanner0, i_scanner1):
        """Transformation between a pair of scanners"""

        beacon0_beacon1_pos = {}
        beacon0_pos = []
        beacon1_pos = []
        beacon0_set = set()
        beacon1_set = set()
        
        # read beacon distance & positions
        beacon_distance_0 = beacon_distance_list[i_scanner0]
        beacon_distance_1 = beacon_distance_list[i_scanner1]
        beacon_position_0 = beacon_position_list[i_scanner0]
        beacon_position_1 = beacon_position_list[i_scanner1]
        
        # shared distance between beacons
        for shared_dist in sorted(set(beacon_distance_0).intersection(set(beacon_distance_1))):
            temp_beacon0 = beacon_distance_0[shared_dist]
            temp_beacon1 = beacon_distance_1[shared_dist]
            beacon0_pos.append(temp_beacon0)
            beacon1_pos.append(temp_beacon1)
            beacon0_set.update(temp_beacon0)
            beacon1_set.update(temp_beacon1)
        
        # find positions of shared beacon
        for beacon0 in sorted(beacon0_set):
            temp_beacon1 = [beacon1_pos[i_b] for i_b, b in enumerate(beacon0_pos) if beacon0 in b]
            beacon1 = set.intersection(*temp_beacon1)

            try:
                beacon0_beacon1_pos[beacon0] = list(beacon1)[0]
            except IndexError:
                temp_counter = collections.Counter(list(itertools.chain.from_iterable(temp_beacon1)))
                temp_counter_max = max(temp_counter.values())
                beacon0_beacon1_pos[beacon0] = [k for k, v in temp_counter.items() if v == temp_counter_max][0]

        beacon0_pos_array = np.array([beacon_position_0[b] for b in sorted(beacon0_beacon1_pos)])
        beacon1_pos_array = np.array([beacon_position_1[beacon0_beacon1_pos[b]] for b in sorted(beacon0_beacon1_pos)])

        # calculate transformations
        transformation_tuple = None
        for i_perspective, perspective in enumerate(perspective_list):
            temp_scanner_pos = set([tuple(b) for b in beacon0_pos_array - np.dot(beacon1_pos_array, perspective)])
            if len(temp_scanner_pos) == 1:
                transformation_tuple = (i_perspective, list(temp_scanner_pos)[0])
                break

        if transformation_tuple is None:
            duplicate_scanner_limit = 2
            while duplicate_scanner_limit < 12 and transformation_tuple is None:
                for i_perspective, perspective in enumerate(perspective_list):
                    temp_scanner_list = [tuple(b) for b in beacon0_pos_array - np.dot(beacon1_pos_array, perspective)]
                    temp_scanner_pos = set(temp_scanner_list)
                    if len(temp_scanner_pos) == duplicate_scanner_limit:
                        temp_scanner_counter = collections.Counter(temp_scanner_list)    
                        transformation_tuple = (i_perspective, [k for k in temp_scanner_pos if temp_scanner_counter[k] == max(temp_scanner_counter.values())][0])
                        break
                duplicate_scanner_limit += 1

        if transformation_tuple is None:
            raise ValueError

        return transformation_tuple

    def transform_all_beacons(last_scanner, this_scanner):
        """Calculate all beacon positions"""

        all_current_beacons = beacon_position_list[this_scanner]
        dfs_dict = nx.algorithms.traversal.depth_first_search.dfs_successors(scanner_graph, 0)
        
        # dfs all scanners
        if this_scanner in dfs_dict:
            for next_scanner in dfs_dict[this_scanner]:
                current_transformation = scanner_transformation(this_scanner, next_scanner)
                temp_beacons = current_transformation[1] + np.dot(transform_all_beacons(this_scanner, next_scanner), perspective_list[current_transformation[0]])
                all_current_beacons += [b for b in temp_beacons]

        all_current_beacons = np.array(sorted(set([tuple(b) for b in all_current_beacons])))

        return all_current_beacons
    
    # main function
    with open(file_path) as f:
        aoc_read = f.read().split('\n\n')

    beacon_distance_list = []
    beacon_position_list = []
    
    # define all 24 perspectives
    temp_perm = [np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
                 np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]]),
                 np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]]),
                 np.array([[-1, 0, 0], [0, -1, 0], [0, 0, 1]]),
                 np.array([[0, -1, 0], [0, 0, -1], [1, 0, 0]]),
                 np.array([[0, 0, -1], [-1, 0, 0], [0, 1, 0]])]
    temp_rot = [np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
                np.array([[1, 0, 0], [0, 0, -1], [0, 1, 0]]),
                np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]]),
                np.array([[1, 0, 0], [0, 0, 1], [0, -1, 0]])]
    perspective_list = [np.dot(p[1], p[0]) for p in itertools.product(temp_perm, temp_rot)]
    
    # read beacon distances for each scanner
    for i_scanner, scanner_read in enumerate(aoc_read):
        beacon_list = [np.array([int(b) for b in beacon.split(',')]) for beacon in scanner_read.split('\n')[1:] if beacon]
        beacon_position_list.append(beacon_list)

        distance_dict = {}
        for i_beacon0 in range(len(beacon_list) - 1):
            for i_beacon1 in range(i_beacon0 + 1, len(beacon_list)):
                temp_distance = np.around(np.sqrt(np.sum(np.square(beacon_list[i_beacon0] - beacon_list[i_beacon1]))), decimals=5)
                distance_dict[temp_distance] = {i_beacon0, i_beacon1}

        beacon_distance_list.append(distance_dict)

    # connect scanners by common beacons
    scanner_graph = nx.Graph()
    for i_scanner0 in range(len(beacon_position_list)):
        for i_scanner1 in range(len(beacon_position_list)):
            if i_scanner0 == i_scanner1:
                continue
            if len(set(beacon_distance_list[i_scanner0]).intersection(set(beacon_distance_list[i_scanner1]))) >= 66:
                scanner_graph.add_edge(i_scanner0, i_scanner1)

    # calculate all scanner positions
    scanner_pos_dict = {0: (0, 0, 0)}
    for scanner in range(1, len(beacon_distance_list)):
        simple_scanner_path = list(nx.algorithms.simple_paths.all_simple_paths(scanner_graph, scanner, 0))[0]
        
        # simple path from scanner 0 to scanner x
        scanner_pos = (0, 0, 0)
        for i_scanner in range(len(simple_scanner_path) - 1):
            temp_transformation = scanner_transformation(simple_scanner_path[i_scanner+1], simple_scanner_path[i_scanner])
            scanner_pos = np.dot(scanner_pos, perspective_list[temp_transformation[0]]) + temp_transformation[1]
        scanner_pos_dict[scanner] = tuple(scanner_pos)
    
    # find max distance
    max_dist = 0
    for i_scanner1 in range(len(beacon_distance_list) - 1):
        for i_scanner2 in range(1, len(beacon_distance_list)):
            max_dist = max(max_dist, int(sum(np.abs(np.array(scanner_pos_dict[i_scanner1]) - np.array(scanner_pos_dict[i_scanner2])))))

    return max_dist

In [6]:
aoc_2021_19_2('example.txt')

3621

In [7]:
aoc_2021_19_2('input.txt')

10864